In [24]:
from gurobipy import Model, GRB

# Data
pensions = [12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 30, 31, 31, 31]
bond_price = [0.98, 0.97, 1.05]
annual_coupon = [0.06, 0.065, 0.075]
maturity_year = [5, 11, 14]
value_at_maturity = [1, 1, 1]
savings_interest = 0.04

# Initialize model
m = Model("PensionFund")

# Decision Variables
x = m.addVars(3, name="x")  # Bonds
s = m.addVars(15, name="s")  # Savings for each year, including start of Year 1 (s_0)

# Objective Function: Minimize the initial investment
m.setObjective(sum(bond_price[j]*x[j] for j in range(3)) + s[0], GRB.MINIMIZE)

# Constraints

# Initial investment (start of Year 1)
# Yearly constraints
for t in range(1, 15):
    inflow = s[t-1] * (1 + savings_interest)  # Principal and interest from previous year's savings
    
    for j in range(3):
        if t < maturity_year[j]:
            inflow += annual_coupon[j] * x[j]
        elif t == maturity_year[j]:
            inflow += (annual_coupon[j] + value_at_maturity[j]) * x[j]

    m.addConstr(inflow == pensions[t-1] + s[t])

# Solving the model
m.optimize()

# Printing the results
if m.status == GRB.Status.OPTIMAL:
    print("Optimal solution found!")
    for j in range(3):
        print(f"Number of Bond {j+1} to purchase: {x[j].x:.2f}")
    print(f"Initial savings amount: ${s[0].x:.2f}K")
else:
    print("No solution found")


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: 12th Gen Intel(R) Core(TM) i9-12900KF, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 14 rows, 18 columns and 58 nonzeros
Model fingerprint: 0x30e48e32
Coefficient statistics:
  Matrix range     [6e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 14 rows, 17 columns, 57 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.171338e+02   0.000000e+00      0s
      10    1.8676840e+02   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.867683968e+02
Optimal solution found!
Number of Bond 1 to purchase: 73.69
Number of Bond 2 to purchase: 77.21
Number of Bond 3 to purchase: 28.84
Initial

In [28]:
73.69 * 980 + 77.21*970 + 28.84*1050 +9380

186771.9